### 사용 라이브러리

In [1]:
import cv2
import numpy as np
import dlib
import math
from datetime import datetime
from datetime import timedelta
import datetime
import time
from collections import Counter

# 사용 함수
- **get_loc_landmarks(landmarks, landmark_idx)**
  - 랜드마크 부위별 좌표값 추출 함수
  - 매개변수
        landmarks:탐지된 얼굴 총 랜드마크
        landmark_idx:landmarks_idx의 key값

- **ratio_closedEyes(loc_left, loc_right, loc_top, loc_btm)**
  - 눈 개폐 비율 계산 함수

- **화면에 텍스트 표시**
  - cvText : 일반
  - cvTextBold : 굵게
  - 매개변수
      frame:표시할 프레임
      text:표시할 글자
      location:텍스트 위치(width,height)


In [118]:
"""
[랜드마크 부위별 좌표값 추출]
*매개변수 -> landmarks:탐지된 얼굴 총 랜드마크, landmark_idx:landmarks_idx의 key값
"""
def get_loc_landmarks(landmarks, landmark_idx) :
    # 랜드마크 점 번호 값 추출
    idx_left, idx_top1, idx_top2, idx_right, idx_btm2, idx_btm1 = landmarks_idxs[landmark_idx]
    
    # 랜드마크 점 위치 값 추출    
    loc_left = (landmarks.part(idx_left).x, landmarks.part(idx_left).y)
    loc_right = (landmarks.part(idx_right).x, landmarks.part(idx_right).y)
    loc_top1 = (landmarks.part(idx_top1).x, landmarks.part(idx_top1).y)
    loc_top2 = (landmarks.part(idx_top2).x, landmarks.part(idx_top2).y)
    loc_btm1 = (landmarks.part(idx_btm1).x, landmarks.part(idx_btm1).y)
    loc_btm2 = (landmarks.part(idx_btm2).x, landmarks.part(idx_btm2).y)

    # 상하단 값 재정의
    loc_top = tuple((round(sum(p)/2) for p in zip(loc_top1, loc_top2)))  # 정중앙 값으로 상단 위치 설정
    loc_btm = tuple((round(sum(p)/2) for p in zip(loc_btm1, loc_btm2)))  # 정중앙 값으로 하단 위치 설정

    # 눈 영역 추출
    region = np.array([loc_left, loc_top1, loc_top2, loc_right, loc_btm2, loc_btm1])
    
    return loc_left, loc_right, loc_top, loc_btm, region
# ---------------------------------------------------------------

"""
[눈 개폐 비율 계산]
"""
def ratio_closedEyes(loc_left, loc_right, loc_top, loc_btm):
    length_horizon = math.hypot(loc_right[0]-loc_left[0], loc_right[1]-loc_left[1])
    length_vertic = math.hypot(loc_top[0]-loc_btm[0], loc_top[1]-loc_btm[1])

    ratio = length_horizon / length_vertic
    
    return ratio
# ---------------------------------------------------------------

"""
[화면에 텍스트 표시]
-cvText : 일반
-cvTextBold : 굵게

*매개변수 -> frame:표시할 프레임, text:표시할 글자, location:텍스트 위치(width,height)
"""
def cvText(frame, text, location):
    font = cv2.FONT_HERSHEY_PLAIN  #폰트 설정
    return cv2.putText(frame, text, location, font, 1, (255, 0, 0), 1, cv2.LINE_AA)
    
def cvTextBold(frame, text, location):
    font = cv2.FONT_HERSHEY_PLAIN  #폰트 설정
    return cv2.putText(frame, text, location, font, 1, (255, 0, 0), 2, cv2.LINE_AA)
# ---------------------------------------------------------------


In [117]:
# dlib 설정
cam = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector() 
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat") # 랜드마크 모델 불러오기
landmarks_idxs = { # 랜드마크 부위별 범위
                   "mouth" : list(range(48, 68)),
                   "right_eyebrow" : list(range(17, 22)),
                   "left_eyebrow" : list(range(22, 27)),
                   "right_eye" : list(range(36, 42)),
                   "left_eye" : list(range(42, 48)),
                   "nose" : list(range(27, 35)),
                   "jaw" : list(range(0, 17))}



# 유저 ID 및 시간 기록
user_id = 'hello'

closeChk_15sec = [0] * 15  # 눈개폐 체크 시간 기준 - 15초

status = 'awake'  # 상태값(awake:깨있음, sleeping:숙면, sleepy:졸음)
sleep_time = {'start':[], 'end':[], 'duration':[]}



while True:
    _, frame = cam.read()    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #프레임 회색조 변경
    faces = detector(gray)
    cvTextBold(frame, f'User: {user_id}', (10,70)) # 화면에 텍스트 표시 함수 호출
    
    for i, face in enumerate(faces):
        num_faces = len(faces)
        landmarks = predictor(gray, face)
        
        """[눈 좌표값 추출]  w/함수get_loc_landmarks"""
        lefteye_left, lefteye_right, lefteye_top, lefteye_btm, lefteye_region = get_loc_landmarks(landmarks, 'left_eye')
        righteye_left, righteye_right, righteye_top, righteye_btm, righteye_region = get_loc_landmarks(landmarks, 'right_eye')

        """[눈 개폐 비율 계산]  w/함수ratio_closedEyes"""
        ratio_left = ratio_closedEyes(lefteye_left, lefteye_right, lefteye_top, lefteye_btm)
        ratio_right = ratio_closedEyes(righteye_left, righteye_right, righteye_top, righteye_btm)
        ratio_close = (ratio_left + ratio_right) / 2

        """
        [눈감음 조건]
        - 눈 개폐비율(ratio_close) > 4.8 => 눈감음 = 1
        [status 조건 설정]
        - 9초 이상 눈감으면 sleeping으로 간주 (눈감음 = 1, 눈뜸 = 0)
        - 5초 단위 구간에서 3초 이상 눈감김이 3번 감지된 경우, sleepy
        """
        if ratio_left >= 4.8 : # 눈감음 조건 설정
            closeChk_15sec.pop(0)
            closeChk_15sec += [1]
        else:
            closeChk_15sec.pop(0)
            closeChk_15sec += [0]            

        # status 조건 설정
        if sum(closeChk_15sec[-10:]) >= 9: # awake->sleeping 조건설정
            if closeChk_15sec[-10:][0] == 0:
                status = 'sleeping'
                sleep_time['start'] += [datetime.datetime.now() - datetime.timedelta(seconds=9)]
            elif closeChk_15sec[-1] == 0: # sleeping->awake 조건설정
                status = 'awake'
                sleep_time['end'] += [datetime.datetime.now() - datetime.timedelta(seconds=1)]
                sleep_time['duration'] += [(sleep_time['end'][-1] - sleep_time['start'][-1]).seconds]
            
        elif (sum(closeChk_15sec[:5]) >= 3) \
             & (sum(closeChk_15sec[5:10]) >= 3) \
             & (sum(closeChk_15sec[10:]) >= 3): # awake->sleepy 조건설정
            status = 'sleepy' 

        else: status = 'awake'

        """[화면에 상태 표시]  w/함수cvText"""
        cvText(frame, f'status: {status}', (10, 90))
        cvText(frame, f'-sleeping {sleep_time["duration"][-3:]}s', (10, 105))

        """
        동공 추출
        x, y, w, h = cv2.boundingRect(lefteye_region)
        mask = np.zeros_like(gray)
        cv2.fillPoly(mask, [lefteye_region], 255)
        eye = cv2.bitwise_and(gray, gray, mask=mask)
        eye = eye[y:y+h, x:x+w]
        _, threshold_eye = cv2.threshold(eye, 70, 255, cv2.THRESH_BINARY)
        cv2.imshow('test', threshold_eye)
        """
        
        cv2.imshow("DON'T SLEEP", frame)

    
    """[탐지 모델 종료 설정]""" 
    if (cv2.waitKey(100) > 0) | (cv2.waitKey(1) & 0xFF == ord('q')):
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()